# Implementando uma CNN para reconhecimento de imagem

_O objetivo desse notebook é implementar uma rede neural convolucional que tem como função reconhecer se a imagem inputada é de um cachorro ou de um gato._

**Grupo: Daniel Augusto, Douglas Abdo, Matheus Fialho e Rennan Haro.**

Ferramentas utilizadas: [TensorFlow](https://www.tensorflow.org), [Keras](https://keras.io). <br>
**Para rodar a rede neural em sua máquina, instale o [Python (3.6+)](https://python.org/download) ou [Anaconda](https://www.anaconda.com/products/individual) e execute o comando `$ pip install -r requirements.txt`** <br>
Desenvolvido e implementado por [Rennan Haro](https://linkedin.com/in/rennanharo).

### Steps para implementaçao da CNN
- Importar as bibliotecas necessárias
- Coletar o dataset
- Separar o dataset (split)
- Setar as varáveis para a construção da CNN
- Estruturar a CNN
- Data augmentation
- Treinar a nossa rede neural
- Testar nosso algoritmo

### Step 1 - Importando as bibliotecas

**Importante: execute o comando `$ pip install -r requirements.txt` antes de prosseguir com esse step.**

In [29]:
# Importando os modulos do Keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

In [28]:
# Importando os modulos adicionais
from IPython.display import display
from PIL import Image
import numpy as np

### Step 2 e 3 - Coletando e splitando o dataset

_Encontre mais informações sobre o dataset e downloads adicionais [neste link](https://www.kaggle.com/c/dogs-vs-cats/data)._

Os datasets estão na pasta `data`. Consistem em basicamente 4 datasets (2 para treinar e 2 para testar), totalizando 2.800 imagens. 1.400 imagens de gatos e 1.400 imagens de cachorros.

**O split do dataset já foi feito. <br>
Temos em nosso dataset de treino 1.000 imagens de cachorros e 1.000 imagens de gatos. <br>
O nosso datset de teste, por sua vez, possui 400 imagens de cada.**

![Imagem](assets/kaggle_dataset.jpeg)

### Step 4 - Setando variáveis (hiperparâmetros) para construção da nossa CNN

In [36]:
img_width, img_height = 150, 150 # Dimensão das imagens

train_data_dir = 'data/train' # Diretório com dados para treino
validation_data_dir = 'data/validation' # Diretório com dados para teste

nb_train_samples = 2000 # Quantidade de samples para treino
nb_validation_samples = 800 # Quantidade de samples para teste
epochs = 50
batch_size = 16

# Setando o shape das imagens inputadas
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

### Step 5 - Construindo a CNN

In [4]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

### Step 6 - Data augmentation

In [5]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 802 images belonging to 2 classes.


### Step 7 - Treinando nossa CNN

In [6]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('first_try.h5')

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
125/125 [==============================] - 35s 278ms/step - loss: 0.7180 - accuracy: 0.5180 - val_loss: 0.6851 - val_accuracy: 0.6425
Epoch 2/50
125/125 [==============================] - 35s 279ms/step - loss: 0.6729 - accuracy: 0.5940 - val_loss: 0.6230 - val_accuracy: 0.6575
Epoch 3/50
125/125 [==============================] - 34s 272ms/step - loss: 0.6408 - accuracy: 0.6505 - val_loss: 0.6077 - val_accuracy: 0.6500
Epoch 4/50
125/125 [==============================] - 36s 285ms/step - loss: 0.6114 - accuracy: 0.6715 - val_loss: 0.5810 - val_accuracy: 0.6750
Epoch 5/50
125/125 [==============================] - 35s 279ms/step - loss: 0.5887 - accuracy: 0.6935 - val_loss: 0.6071 - val_accuracy: 0.6925
Epoch 6/50
125/125 [==============================] - 35s 283ms/step - loss: 0.5795 - accuracy: 0.7065 - val_loss: 0.6443 - val_accuracy: 0.6787
Epoch 7/50
125/125 [==============================] - 

### Step 8 - Testando nossa CNN

In [39]:
test_image = image.load_img('random.png', target_size = (150, 150))
#test_image = image.load_img('data/validation/dogs/dog.10500.jpg', target_size = (150, 150))
#test_image = image.load_img('data/validation/cats/cat.1000.jpg', target_size = (150, 150))
test_image = image.img_to_array(test_image)
test_image = test_image/.255
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)

train_generator.class_indices
if result[0][0] >= 0.5:
    prediction = 'dog'
else:
    prediction = 'cat'
    
print(prediction)

dog
